In [15]:
import re
import string
import contractions
import gensim
from gensim.models import Word2Vec
import warnings
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from scipy.sparse import save_npz, load_npz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils.class_weight import compute_class_weight

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import expon, reciprocal

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#from spellchecker import SpellChecker

In [2]:
# pandas with tqdm
tqdm.pandas()

nltk.download("wordnet")

nltk.download("stopwords")

nltk.download('punkt')

nltk.download('omw-1.4')

#! warnings.filterwarnings("ignore", category=ConvergenceWarning)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
train = pd.read_csv('data/train.csv')

In [4]:
train = train[['Review', 'overall']]

In [5]:
train = train.dropna(subset=['Review'])

In [6]:
# read processed_train from pickle
processed_train = pd.read_pickle("processed_train.pkl")

In [7]:
processed_train = processed_train[["tokenized", "tokenized_joined", "overall"]]

In [8]:
tokenized_texts = processed_train['tokenized'].tolist()

skipgram = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, sg=1, min_count=2)

In [24]:
skipgram.save("skipgram.model")

In [25]:
vocab_size = len(skipgram.wv.key_to_index) + 1  # Adding 1 to account for padding token
embedding_dim = 100  # Or the vector_size you used for your Skip-gram model

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in skipgram.wv.key_to_index.items():
    embedding_vector = skipgram.wv[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# save embedding_matrix to pickle
with open("embedding_matrix.pkl", "wb") as f:
    pickle.dump(embedding_matrix, f)

In [27]:
embedding_matrix.shape[0]

26898

In [26]:
vocab_size

26898

In [10]:
# Initialize and fit the tokenizer
tokenizer = Tokenizer(num_words=vocab_size-1)  # vocab_size minus 1 to account for padding token
tokenizer.fit_on_texts(processed_train['tokenized_joined'].tolist())

# Convert texts to sequences of integers
sequences = tokenizer.texts_to_sequences(processed_train['tokenized_joined'].tolist())

# Pad sequences to ensure uniform length
max_length = max(len(seq) for seq in sequences)  # Or choose a max_length that suits your dataset
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [19]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, trainable=True))
model.add(SpatialDropout1D(0.2))  # Helps prevent overfitting by dropping entire 1D feature maps
model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # For 5 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
# check model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1735, 100)         2689800   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 1735, 100)        0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 1735, 256)        234496    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                      

In [21]:
y = processed_train['overall'] - 1

In [22]:
classes = np.unique(y)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weights_dict = {class_label: weight for class_label, weight in zip(classes, class_weights)}

In [23]:
history = model.fit(padded_sequences, y, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights_dict)

Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_3/bidirectional_3/forward_lstm_5/while/lstm_cell_12/split' defined at (most recent call last):
    File "c:\Users\dhruv\anaconda3\envs\TF\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\dhruv\anaconda3\envs\TF\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\dhruv\anaconda3\envs\TF\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\dhruv\anaconda3\envs\TF\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "c:\Users\dhruv\anaconda3\envs\TF\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\Users\dhruv\anaconda3\envs\TF\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\User